In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("PySpark").getOrCreate()

In [2]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [3]:
# Get unique data
emp_unique = emp.distinct()

In [4]:
emp_unique.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [5]:
# Get unique dept id's
emp_unique_dept = emp_unique.select("department_id").distinct()

In [6]:
emp_unique_dept.show()

+-------------+
|department_id|
+-------------+
|          101|
|          102|
|          103|
|          104|
|          105|
|          106|
|          107|
+-------------+



In [7]:
# Window functions
from pyspark.sql.window import Window
from pyspark.sql.functions import max,col,desc

window_spec = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())
max_func = max(col("salary")).over(window_spec)
emp_max = emp_unique.withColumn("max_salary",max_func)

In [8]:
emp_max.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|max_salary|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|     70000|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|     70000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|     70000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|     55000|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|     55000|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|     55000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|     55000|
|        019|          103|  Steven Chen| 36|  Male| 62000|2015-08-01|     62000|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|     62000|
|        009|   

In [10]:
# 2nd highest salary of each department
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, desc

second_highest_spec = Window.partitionBy("department_id").orderBy(col("salary").desc())
row_number_func = row_number().over(second_highest_spec)
second_highest_salary = emp_unique.withColumn("second_highest_salary",row_number_func).where("second_highest_salary = 2")

In [11]:
second_highest_salary.show()

+-----------+-------------+-----------+---+------+------+----------+---------------------+
|employee_id|department_id|       name|age|gender|salary| hire_date|second_highest_salary|
+-----------+-------------+-----------+---+------+------+----------+---------------------+
|        001|          101|   John Doe| 30|  Male| 50000|2015-01-01|                    2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|                    2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|                    2|
|        018|          104|  Nancy Liu| 29|      | 50000|2017-06-01|                    2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|                    2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|                    2|
|        014|          107|  Emily Lee| 26|Female| 46000|2019-01-01|                    2|
+-----------+-------------+-----------+---+------+------+----------+---------------------+

In [13]:
# Window function using expr
from pyspark.sql.functions import expr
emp_3 = emp.withColumn("rn",expr("ROW_NUMBER() OVER(PARTITION BY department_id ORDER BY salary DESC)")).where("rn = 2")
emp_3.show()

+-----------+-------------+-----------+---+------+------+----------+---+
|employee_id|department_id|       name|age|gender|salary| hire_date| rn|
+-----------+-------------+-----------+---+------+------+----------+---+
|        001|          101|   John Doe| 30|  Male| 50000|2015-01-01|  2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|  2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|  2|
|        018|          104|  Nancy Liu| 29|      | 50000|2017-06-01|  2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|  2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|  2|
|        014|          107|  Emily Lee| 26|Female| 46000|2019-01-01|  2|
+-----------+-------------+-----------+---+------+------+----------+---+

